In [65]:
import pandas as pd
import os
import json
import data_wrangler
## Remark : importing data_wrangler preprocesses and creates the dataset containing the characters of every movie in the dataset

df = pd.read_csv("processed_data.csv")

Gathering the 10 biggest revenue-inducing movies each year

In [68]:
# Creating a new column for the release year (without month and day) of the movie$
df['Release_year'] = df['Release_date'].str[:4]

# Ditching entries with nan revenue 
df = df.dropna(subset=['Revenue'])


def topTenByYear(year):
    """
    Returns all characters of the top ten movies by revenue in a given year.
    """
    df_year = df[df['Release_year'] == year]
    only_movies = df_year.drop_duplicates(subset=['Wikipedia_movie_ID', 'Revenue'])
    top10 = only_movies.nlargest(10, 'Revenue')

    return df_year[df_year['Wikipedia_movie_ID'].isin(top10['Wikipedia_movie_ID'])]

def isInTopTen(name, year):
    """
    Checks if a name was given to a character in a movie that was in the top ten by revenue in a given year.
    If yes, return the corresponding dataset entry, otherwise return None.
    """
    top10 = topTenByYear(year)
    return top10[top10['Character_name'] == name]

isInTopTen("Neo", "1999")



,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Release_date,Revenue,Runtime,Languages,Countries,Genres,Character_name,Actor_DOB,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age,Freebase_character_map,Freebase character ID,Freebase actor ID,Release_year
319733,30007,/m/07cz2,The Matrix,1999-03-31,463517383.0,135.0,English,"United States of America, Australia","Thriller, Science Fiction, Adventure, Doomsday...",Neo,1964-09-02,M,1.86,/m/03bkbh,Keanu Reeves,34.0,/m/0k5wyl,/m/01hmvs,/m/0479b,1999
